# Analyse References of Paper to Find an Answer

Following example shows how to extract references from the document and do a semantic search over the documents and all its references

Optionally, start Summa server to enhance performance of queries to STC

In [ ]:
import logging
import sys

from stc_geck.advices import BaseDocumentHolder
from cybrex.cybrex_ai import CybrexAI

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

cybrex = CybrexAI()
query = 'What drugs are used for haemophilia treatment?'
seed_doi = '10.1111/j.1365-2516.2007.01474.x'
await cybrex.start()

In [ ]:
documents = await cybrex.get_documents(f'id.dois:{seed_doi}', n_documents=1)
seed_document = documents[0].document
related_document_ids = {f'id.dois:{seed_doi}'}

# Get out references
for reference in seed_document['references']:
    related_document_ids.add(f'id.dois:{reference["doi"]}')

# Get in references
referencing_documents = await cybrex.get_documents(f'rd:{seed_doi})', n_documents=100)
for referencing_document in referencing_documents:
    referencing_document_holder = BaseDocumentHolder(referencing_document)
    related_document_ids.add(referencing_document_holder.get_internal_id())

print('Following documents will be queries:', related_document_ids)
related_documents = await cybrex.get_documents(' '.join(related_document_ids), n_documents=100)

In [ ]:
related_chunks = await cybrex.semantic_search_in_documents(query, related_documents, n_chunks=10, minimum_score=0.5, skip_downloading_pdf=False)

In [ ]:
related_chunks[:10]